In [2]:
import training
import torch
import OBC.networks
import argparse
import data_loader as dl
import torch.optim as optim
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as f
import torchvision
from torchvision import transforms, datasets
from datetime import datetime
import numpy as np
import OBC.ClassificationMetric as ClassificationMetric
import PoseEstimation.PELoss as PELoss
from PoseEstimation.LinemodDataset import LinemodDataset


### Define the network and the loader  and paramters

In [3]:
CLASSES = 2
batch = 4
lr = 1e-3
momentum = 0.9
decay = 0
step = 100

model = OBC.networks.resnet18(CLASSES+4, None)

folder = '/home/fcdl/Develop/Data/linemod'

# Build the training loader
train_loader = dl.get_image_folder_loaders(folder + "/sample", LinemodDataset, "NO", batch)


params_to_optim = list(filter(lambda p: p.requires_grad, model.parameters()))
optimizer = optim.SGD(params_to_optim, lr=lr, momentum=momentum, weight_decay=decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step)

cost_function = PELoss.PE3DLoss(2)

### Perform a sample training (only one batch)

In [4]:
# Training part
model.train()
losses = 0
current = 0

# Perform the training procedure
(data, target) =  next(iter(train_loader))
print(target.size())
optimizer.zero_grad()
output = model(data)
loss = cost_function(output, target)
loss.backward()

optimizer.step()

losses += loss.item()
current += 1
    
print(losses/current)

torch.Size([4, 5])
3.584178924560547


### Compute the loss (get input from previous output)
This is the code of PE3DLoss' forward method in PELoss (with some slight modification)

In [5]:
# print("Output")
# print(output)
# print("Target")
# print(target)
x = output
class_input, rot_input = x[:, 0:CLASSES], x[:, CLASSES:]
class_target, rot_target = target[:, 0], target[:, 1:]

# Quaternion / quaternion_norm  (L2 regularization)
rot_input = f.normalize(rot_input, p=2, dim=1)

print("rot_target")
print(rot_target)
print("rot_input")
print(rot_input)

cel = nn.CrossEntropyLoss()
pwd = PELoss.GeodesicDistance()
ce_loss = cel(class_input, class_target.long())
distance = pwd(rot_input, rot_target)
final_loss = ce_loss + distance.mean()

print("Geod. dist= " + str(distance.mean()))
print("Cross loss= " + str(ce_loss))

rot_target
tensor([[ 0.9609, -0.2749, -0.0256,  0.0208],
        [ 0.9358, -0.3506, -0.0113, -0.0363],
        [ 0.9659, -0.2556, -0.0118, -0.0385],
        [-0.6175,  0.2274,  0.2999,  0.6906]])
rot_input
tensor([[ 0.0437,  0.4942,  0.8024, -0.3317],
        [ 0.2517, -0.0527,  0.8852, -0.3878],
        [ 0.0667,  0.1596,  0.8943, -0.4126],
        [ 0.4774,  0.4077,  0.7581, -0.1764]])
Geod. dist= tensor(2.8875)
Cross loss= tensor(0.6967)


### Compute the Metric (get input from previous output)
This is the code of PEMetric's forward method in PELoss (with some slight modification)

In [9]:
from PoseEstimation.utils import rotation_equals
import math

x = output
        
class_input, rot_input = x[:, 0:2], x[:, 2:]
class_target, rot_target = target[:, 0].long(), target[:, 1:]

# Quaternion / quaternion_norm  (L2 regularization)
rot_input = f.normalize(rot_input, p=2, dim=1)

pred = torch.max(class_input, 1)[1]
correct_class = pred.eq(class_target.data.view_as(pred))

correct_pose = rotation_equals(rot_input, rot_target, math.radians(10))

correct = (correct_class & correct_pose).cpu()  # to be correct both must be correct [1 and 1]

print(correct_class)
print(correct_pose)
correct.sum(-1)

tensor([ 1,  1,  1,  0], dtype=torch.uint8)
tensor([ 0,  0,  0,  0], dtype=torch.uint8)


tensor(0)

In [16]:
# Code to check if a quaternions batch is unit norm.
rott = torch.tensor([[-0.3837, -0.1108,  0.6290,  0.6670],
        [-0.1176, -0.3606,  0.8805,  0.2843],
        [ 0.2009, -0.7628,  0.3014,  0.5357],
        [-0.6261, -0.1755,  0.3599,  0.6691]])
print(torch.norm(rott, 2, 1))





tensor([ 1.0000,  1.0000,  1.0000,  1.0000])
